In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

def load_and_preprocess_data(file_path):
    column_names = ['class', 'T3', 'T4U', 'T4', 'FTI', 'TSH']
    data = pd.read_csv(file_path, names=column_names, header=None)

    le = LabelEncoder()
    data['class'] = le.fit_transform(data['class'])

    for col in data.columns:
        if col != 'class':
            data[col] = pd.to_numeric(data[col], errors='coerce')
    data = data.dropna()

    return data, le

file_path = 'https://www.dropbox.com/scl/fi/pvtlqgewakfacyiy9u3dc/new-thyroid.data?rlkey=h133b1idt3v7m1fgr1x210kin&st=463tr40p&dl=1'
data, le = load_and_preprocess_data(file_path)

X = data.drop('class', axis=1)
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(5,)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

def predict(T3, T4, TSH, TT4, FTI):
    input_data = np.array([[T3, T4, TSH, TT4, FTI]])
    prediction = model.predict(input_data)
    return np.argmax(prediction)

def get_input_and_predict():
    print("Enter the following values:")
    T3 = float(input("T3: "))
    T4 = float(input("T4: "))
    TSH = float(input("TSH: "))
    TT4 = float(input("TT4: "))
    FTI = float(input("FTI: "))

    prediction = predict(T3, T4, TSH, TT4, FTI)
    print(f"Predicted class: {le.inverse_transform([prediction])[0]}")

get_input_and_predict()